In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# LDDMM landmark stochastic dynamics

In [ ]:
from src.manifolds.landmarks import *
M = landmarks(6)
print(M)

from src.plotting import *

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

In [ ]:
# example configuration
M.k_sigma.set_value(tensor(np.diag((.5,.5))))

q = M.coordsf(np.vstack((np.linspace(-.5,.5,M.N.eval()),np.zeros(M.N.eval()))).T.flatten())
v = tensor(np.vstack((np.zeros(M.N.eval()),np.ones(M.N.eval()))).T.flatten())
p = M.flatf(q,v)
print("q = ", q)
print("p = ", p)

## Brownian Motion

In [ ]:
# coordinate form
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

n_steps.set_value(1000)
(ts,xs,charts) = M.Brownian_coordsf(q,dWsf(M.dim.eval()))

# plot
M.plot()
M.plot_path(zip(xs,charts))
plt.show()

# plot multiple sample paths
N = 5
xss = tensor(np.zeros((N,n_steps.eval(),M.dim.eval())))
chartss = tensor(np.zeros((N,n_steps.eval(),q[1].shape[0])))
for i in range(N):
    (ts,xs,charts) = M.Brownian_coordsf(q,dWsf(M.dim.eval()))
    xss[i] = xs
    chartss[i] = charts

# plot
M.newfig()
M.plot()
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, N)]
for i in range(N):
    M.plot_path(zip(xss[i],chartss[i]),color=colors[i])
M.plotx(q,color='r')
plt.show()

# Langevin equations
see https://arxiv.org/abs/1605.09276

In [ ]:
from src.stochastics import Langevin
Langevin.initialize(M)

dWs = dWsf(M.dim.eval())

(ts,qps,charts) = M.Langevinf(q,p,.5,.25,dWs)
M.plot()
M.plot_path(zip(qps[:,0,:],charts))
plt.axis('off')
plt.savefig('Langevin.pdf')
plt.show()

## Stochastic EPDiff / Eulerian

In [ ]:
# define noise field grid
minx = -1; maxx = 1
miny = -1; maxy = 1
pts = 3

X, Y = np.meshgrid(np.linspace(minx,maxx,pts),np.linspace(miny,maxy,pts))
xy = np.vstack([X.ravel(), Y.ravel()]).T
sigmas_x = np.hstack((xy,xy)).reshape((-1,2))
sigmas_a = .05*np.tile(np.eye(2),(sigmas_x.shape[0]//2,1))
#sigmas_x = np.array([[0.,0.]])
#sigmas_a = np.array([[.1,0.]])
J = sigmas_x.shape[0]

# noise kernels
k_alpha = 1.
k_sigma = np.diag(((maxx-minx)/(pts-1),(maxy-miny)/(pts-1)))
inv_k_sigma = theano.tensor.nlinalg.MatrixInverse()(k_sigma)
k = lambda x: k_alpha*T.exp(-.5*T.sqr(T.tensordot(x,inv_k_sigma,(0 if x.type == T.vector().type else 2,1))).sum(0 if x.type == T.vector().type else 2))

# initialize with specified noise kernel
from src.stochastics import Eulerian
Eulerian.initialize(M,k=k)

# integrate
(ts,qps,charts) = M.Eulerianf(q,p,sigmas_x.flatten(),sigmas_a.flatten(),dWsf(J))
M.plot()
M.plot_path(zip(qps[:,0,:],charts))
plt.axis('off')
plt.plot(sigmas_x[:,0],sigmas_x[:,1],'x')
plt.savefig('Eulerian.pdf')
plt.show()

## Bridge Sampling

In [ ]:
# Delyon/Hu guided process
from src.stochastics.guided_process import *

# guide function
Cholesky = T.slinalg.Cholesky()
phi = lambda q,v: T.tensordot(T.nlinalg.MatrixInverse()(Cholesky(M.gsharp(q))),-(q[0]-v).flatten(),(1,0))

# target shape
v = M.coordsf(np.vstack((np.linspace(-.5,.5,M.N.eval()),np.ones(M.N.eval()))).T.flatten())

(Brownian_coords_guided,Brownian_coords_guidedf) = get_guided_likelihood(
    M,M.sde_Brownian_coords,phi,lambda q: Cholesky(M.gsharp(q)),use_charts=True)

# note that the noise is scaled by .2 (i.e. .2*dWs)
(ts,qs,charts,log_likelihood,log_varphi) = Brownian_coords_guidedf(q,v[0],.2*dWsf(M.dim.eval()))[:5]
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])

# Visualize bridge
M.plot()
M.plot_path((q,)+tuple(zip(qs,charts)),curve=True) # no translation added
M.plotx(v,color='k',curve=True)
plt.axis('off')
# plt.savefig('bridge.pdf')

## Stochastic Development

In [ ]:
# # development dynamics
# from src.framebundle import FM
# from src.stochastics import stochastic_development
# FM.initialize(M)
# stochastic_development.initialize(M)

# n_steps.set_value(100)

In [ ]:
# # landmarks
# x = q
# # frame
# nu = np.linalg.cholesky(M.gsharpf(x))
# # frame bundle element
# u = np.concatenate((x,nu.flatten()))

In [ ]:
# # stochastic development
# w = dWsf(M.dim.eval()) # noise / anti-development
# start = time.time()
# (ts,us) = M.stochastic_developmentf(u,w)
# print(time.time() - start)

# # plot with frame
# M.plot()
# M.plotFMx(us,N_vec=5)
# plt.show()

# # plot only trajectory
# plt.figure()
# M.plot()
# M.plotx(us[:,0:M.dim.eval()])
# plt.show()

# # plot noise / anti-development
# plt.figure()
# plt.plot(w[:,0],w[:,1])
# plt.axis('equal')
# plt.show()